In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re 

#from petdata_functions import CleanPetData
#from petdata_functions import ReformatDates


#read in data 
chidat = pd.read_csv("Chicago_adopted_20k_Jan_21.csv") 
chidat['City'] = 'Chicago'

houstdat = pd.read_csv("Houston_adopted_20k_Jan_21.csv") 
houstdat['City'] = 'Houston'

elpdat = pd.read_csv("ElPaso_adopted_20k_Jan_21.csv") 
elpdat['City'] = 'ElPaso'

Minndat = pd.read_csv("Chicago_adopted_20k_Jan_21.csv") 
Minndat['City'] = 'Minne'

dendat = pd.read_csv("Denver_adopted_20k_Jan_21.csv") 
dendat['City'] = 'Denver'

stldat = pd.read_csv("StL_adopted_20k_Jan_21.csv")
stldat['City'] = 'StLouis'

inddat = pd.read_csv("Indy_adopted_20k_Jan_21.csv")
inddat['City'] = 'Indy'

petdat = pd.concat([chidat,houstdat,elpdat,Minndat,dendat, stldat, inddat])

#print(petdat.iloc[0]['photos'])
#print(list(petdat.columns))


#reformat dates and calculate difference in dates (= target variable)
fmt ='%Y-%m-%dT%H:%M:%S+0000'
petdat['published_at'] =  pd.to_datetime(petdat['published_at'], format=fmt)
petdat['status_changed_at'] =  pd.to_datetime(petdat['status_changed_at'], format=fmt)
petdat['time_diff'] = petdat['status_changed_at'] - petdat['published_at']
petdat['target_hour'] = petdat.time_diff.astype('timedelta64[h]')
petdat['target_day'] = petdat.time_diff.astype('timedelta64[D]')
petdat['log_target_hour'] = np.log(petdat['target_hour'])

#calculate number of tags
petdat['num_tags']= petdat['tags'].str.count(',', re.I) 

#calculate number of photos
petdat['num_photos']= petdat['photos'].str.count('small', re.I) 

#calculate number of words in description 
petdat['num_words_desc']= petdat['description'].str.count(' ', re.I) 

#calculate if adopted > 1 month 
petdat['bin_month_target'] = np.where(petdat['target_day']>30, 0, 1)

print(petdat.head())


   Unnamed: 0        id organization_id  \
0           0  47180520            IL53   
1           1  47171077           IL496   
2           2  47160275           IL922   
3           3  47160277           IL922   
4           4  47159557           IL274   

                                                 url type species    age  \
0  https://www.petfinder.com/dog/gwen-47180520/il...  Dog     Dog  Adult   
1  https://www.petfinder.com/dog/keeper-47171077/...  Dog     Dog  Adult   
2  https://www.petfinder.com/dog/kano-47160275/il...  Dog     Dog  Young   
3  https://www.petfinder.com/dog/bernie-47160277/...  Dog     Dog  Adult   
4  https://www.petfinder.com/dog/mr-snuggles-4715...  Dog     Dog  Young   

   gender    size    coat  ... _links.organization.href     City time_diff  \
0  Female   Large     NaN  ...   /v2/organizations/il53  Chicago  00:00:00   
1    Male  Medium  Medium  ...  /v2/organizations/il496  Chicago  19:24:54   
2    Male  Medium     NaN  ...  /v2/organizations/

In [18]:
import sklearn
import pickle
from sklearn.linear_model import LogisticRegression

traindat = petdat[['bin_month_target',
                    'age',
                   #'organization_id',
                   #'contact.address.city',
                   'City',
                   'size',
                   'coat',
                   #'distance',
                   #'colors.primary',
                   #'colors.secondary',
                   #'breeds.primary',
                   #'breeds.secondary',
                   #'breeds.mixed',
                   #'breeds.unknown',
                  'attributes.spayed_neutered',
                   #'attributes.house_trained',
                  'attributes.special_needs']]
                  # 'attributes.shots_current',
                  # 'environment.cats',
                  # 'environment.children',
                  # 'environment.dogs',
                  # 'bin_month_target']]

#make a df 'X' of features to be used  as predictors in the model, right now all of traindat
X = traindat.drop(columns = ['bin_month_target'])

#one hot coding for variables
cols_to_transform = ['age',
                     'coat',
                     'City',
                     'size',
                    'attributes.spayed_neutered',
                    'attributes.special_needs']

X = pd.get_dummies(X, columns = cols_to_transform)

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X, traindat.bin_month_target, test_size=0.20, random_state=42)


logisticRegr = LogisticRegression()
logisticRegr.fit(X_train,y_train)
my_y_pred = logisticRegr.predict(X_test)

print(sklearn.metrics.f1_score(y_true=y_test, y_pred = my_y_pred))

logisticRegr.score(X_test, y_test)




0.8076363636363636


0.7042332268370607